In [170]:
# load relevant packages, set some display parameters
%reset -f

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RepeatedKFold
from math import sqrt
from scipy import stats
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, learning_curve
# import ifsg
import glob, os #I ADDED
from sklearn.svm import SVR, LinearSVR #I ADDED
from sklearn.tree import DecisionTreeRegressor #I ADDED
import pickle
import re
import random
import os
import warnings
import seaborn as sns
import gc
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

#ifsg.__version__

In [171]:
# load data
path_dataset = 'W:\\Work\\Assense\\ML\\chars_file'
observation_filename = os.path.join(path_dataset,'updates_chars_file_04_03.csv')
observation_table = pd.read_csv(observation_filename)#,usecols=range(9))#, index_col=0,skiprows=[0,1,2])#,usecols=range(12))
set(observation_table.company_name)

{'Actiview', 'Cola', 'Deloitte', 'Dts', 'Jsol', 'Mitsubishi', 'Ntt'}

In [172]:
observation_table.replace({'Precision':'Thoroughness',
                     'Decisiveness Vs. Hesitant':'Decisiveness',
                     'Purposefulness vs. Hesitant':'Decisiveness',
                     'Monitor and review _ process and product Monitoring':'Self Monitoring',
                     'Monitor And Review | Process And Product Monitoring':'Self Monitoring',
                     'Monitor and review – process and product Monitoring':'Self Monitoring',
                     'Quality Vs. Quantity':'Quantity vs Quality',
                     'Time management':'Time Management',
                     '(stress tolerance) Reaction to stress and difficulty':'Stress Tolerance',
                     'Stress Tolerance | Reaction To Stress And Difficulty':'Stress Tolerance'},inplace=True)
observation_table.head()


,id,company_name,Purposefulness Vs. Hesitant,Time Management,Verbal reasoning,Collegial Behavior,Adaptability to Changes,Performance IQ,Emotional Understanding,Spatial reasoning,Processing Speed,Appropriate Behaviour Understanding,Cognitive ability total score,Goal orientation,Orderliness,Performance under stress,Thoroughness,Agreeableness,Memory,Verbal Reasoning,Attention: Coping with Distractions,Quantitative Reasoning,Processing speed,Intelligence Quotient,Methodicalness,Self-efficacy,Working memory,Quantitative reasoning,Performance under Stress,Performance IQ,Stress Tolerance,Empathy,Conscientiousness,Planning,Perceptual Organization,Self Monitoring,Monitoring,Coping with stress,Teamwork,Decisiveness,Adaptability to changes,Risk Taking,Precision,Quantity vs Quality
0,5beadc3169056d543c06c4e66,Actiview,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN
1,12345678,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5C45C837378D814141D7B22C,Actiview,NaN,NaN,NaN,NaN,NaN,6.0,NaN,6.0,6.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5b421df4ff36d1e2d5a70e77,Actiview,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,5.0,NaN,3.0,NaN,3.0,NaN,NaN,NaN,6.0,NaN
4,5bb3c16069ad6d50f0356ab7,Actiview,4.0,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,3.0,NaN


In [119]:
company  ='Jsol'
observation_table = observation_table[observation_table.company_name==company]
# observation_table

In [161]:
observation_table = observation_table[~(observation_table.company_name.str.contains('Actiview|Deloitte|Cola'))]
# observation_table

In [173]:
observation_table = observation_table.dropna(axis=1,how = 'all')
observation_table.replace(to_replace= -1, value = np.nan, inplace = True)
observation_table.head()
# observation_table

,id,company_name,Purposefulness Vs. Hesitant,Time Management,Verbal reasoning,Collegial Behavior,Adaptability to Changes,Performance IQ,Emotional Understanding,Spatial reasoning,Processing Speed,Appropriate Behaviour Understanding,Cognitive ability total score,Goal orientation,Orderliness,Performance under stress,Thoroughness,Agreeableness,Memory,Verbal Reasoning,Attention: Coping with Distractions,Quantitative Reasoning,Processing speed,Intelligence Quotient,Methodicalness,Self-efficacy,Working memory,Quantitative reasoning,Performance under Stress,Performance IQ,Stress Tolerance,Empathy,Conscientiousness,Planning,Perceptual Organization,Self Monitoring,Monitoring,Coping with stress,Teamwork,Decisiveness,Adaptability to changes,Risk Taking,Precision,Quantity vs Quality
0,5beadc3169056d543c06c4e66,Actiview,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN
1,12345678,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5C45C837378D814141D7B22C,Actiview,NaN,NaN,NaN,NaN,NaN,6.0,NaN,6.0,6.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5b421df4ff36d1e2d5a70e77,Actiview,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.0,5.0,NaN,3.0,NaN,3.0,NaN,NaN,NaN,6.0,NaN
4,5bb3c16069ad6d50f0356ab7,Actiview,4.0,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,NaN,NaN,3.0,NaN


In [174]:
observation_table = observation_table[['company_name',
                                       'Cognitive ability total score','Performance IQ','Spatial reasoning','Verbal Reasoning',
                                       'Processing Speed','Quantitative Reasoning','Self Monitoring','Decisiveness','Orderliness',
                                       'Goal orientation', 'Methodicalness','Planning','Risk Taking','Thoroughness','Time Management',
                                       'Performance under Stress', 'Adaptability to changes', 'Appropriate Behaviour Understanding',
                                       'Attention: Coping with Distractions', 'Conscientiousness','Collegial Behavior','Teamwork',
                                       'Emotional Understanding',"Empathy"]]

In [179]:
observation_table.tail(20)

,Adaptability to changes,Appropriate Behaviour Understanding,Attention: Coping with Distractions,Cognitive ability total score,Collegial Behavior,Conscientiousness,Decisiveness,Emotional Understanding,Empathy,Goal orientation,Methodicalness,Orderliness,Performance IQ,Performance under Stress,Planning,Processing Speed,Quantitative Reasoning,Risk Taking,Self Monitoring,Spatial reasoning,Teamwork,Thoroughness,Time Management,Verbal Reasoning,company_name
737,NaN,5.300000,5.000000,4.800000,4.000000,4.000000,4.600000,5.000000,3.500000,5.000000,5.000000,NaN,5.000000,NaN,4.300000,NaN,NaN,5.000000,3.400000,5.000000,4.700000,3.400000,4.600000,NaN,Cola
738,NaN,3.700000,5.000000,4.900000,4.600000,5.000000,4.400000,6.000000,5.100000,5.000000,5.000000,NaN,4.900000,NaN,3.700000,NaN,NaN,5.000000,3.200000,4.300000,3.800000,3.100000,4.800000,NaN,Cola
739,NaN,NaN,5.000000,2.600000,NaN,NaN,3.000000,NaN,NaN,5.000000,5.000000,NaN,3.300000,NaN,2.700000,NaN,NaN,NaN,3.400000,2.500000,NaN,2.600000,2.700000,NaN,Cola
740,NaN,NaN,5.000000,3.400000,NaN,NaN,3.600000,NaN,NaN,5.000000,5.000000,NaN,3.500000,NaN,3.500000,NaN,NaN,NaN,3.400000,3.300000,NaN,3.200000,3.700000,NaN,Cola
741,NaN,NaN,5.000000,4.600000,NaN,NaN,4.500000,NaN,NaN,5.000000,5.000000,NaN,5.100000,NaN,3.300000,NaN,NaN,NaN,3.000000,4.700000,NaN,3.000000,4.500000,NaN,Cola
742,NaN,NaN,5.000000,4.600000,NaN,NaN,4.500000,NaN,NaN,5.000000,5.000000,NaN,5.100000,NaN,3.300000,NaN,NaN,NaN,3.000000,4.700000,NaN,3.000000,4.500000,NaN,Cola
743,NaN,2.800000,5.500000,4.700000,5.200000,4.700000,3.900000,6.000000,5.400000,4.500000,5.000000,NaN,4.300000,NaN,5.400000,NaN,NaN,2.500000,4.100000,4.500000,4.500000,5.300000,4.700000,NaN,Cola
744,NaN,NaN,5.000000,5.100000,NaN,NaN,4.200000,NaN,NaN,5.000000,5.000000,NaN,5.000000,NaN,5.900000,NaN,NaN,NaN,4.200000,5.000000,NaN,5.500000,5.400000,NaN,Cola
745,NaN,NaN,5.000000,4.600000,NaN,NaN,4.600000,NaN,NaN,5.000000,5.000000,NaN,4.900000,NaN,4.100000,NaN,NaN,NaN,2.900000,4.800000,NaN,4.400000,4.500000,NaN,Cola
746,NaN,NaN,5.000000,4.900000,NaN,NaN,4.900000,NaN,NaN,5.000000,5.000000,NaN,5.100000,NaN,4.400000,NaN,NaN,NaN,2.900000,5.200000,NaN,4.400000,4.600000,NaN,Cola


In [178]:
# look at specific company
describe_table = observation_table.describe()
observation_table = observation_table.append(describe_table)
# observation_table

In [180]:
observation_table = observation_table[['company_name',
                                       'Cognitive ability total score','Performance IQ','Spatial reasoning','Verbal Reasoning',
                                       'Processing Speed','Quantitative Reasoning','Self Monitoring','Decisiveness','Orderliness',
                                       'Goal orientation', 'Methodicalness','Planning','Risk Taking','Thoroughness','Time Management',
                                       'Performance under Stress', 'Adaptability to changes', 'Appropriate Behaviour Understanding',
                                       'Attention: Coping with Distractions', 'Conscientiousness','Collegial Behavior','Teamwork',
                                       'Emotional Understanding',"Empathy"]]

In [181]:
# observation_table

In [ ]:
plt.figure(figsize=(20,30))
for i,col in enumerate(observation_table.columns):
    plt.subplot(8,4,i+1)
    observation_table[col].hist()
    plt.title(col)


In [182]:
observation_table.to_excel('tot_observation.xlsx')

0